In [1]:
!export LC_ALL=C.UTF-8
!export LANG=C.UTF-8
!export LANGUAGE=C.UTF-8

In [2]:
!pip install -U torch torchvision torchaudio
!pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
!rsync -ah --progress /content/drive/MyDrive/dataset/ /content/dataset/

ストリーミング出力は最後の 5000 行に切り捨てられました。
          1.80K 100%    0.00kB/s    0:00:00 (xfr#1489, to-chk=2499/5944)
labels/train/166233fa-Ch120241207233229.txt
            900 100%    1.35kB/s    0:00:00 (xfr#1490, to-chk=2498/5944)
labels/train/16a14d1e-Ch120241210094741.txt
            547 100%    0.40kB/s    0:00:01 (xfr#1491, to-chk=2497/5944)
labels/train/16cce3bd-Ch120241207094305.txt
            812 100%    0.00kB/s    0:00:00 (xfr#1492, to-chk=2496/5944)
labels/train/16e46f5a-Ch120241207120748.txt
          1.96K 100%    2.86kB/s    0:00:00 (xfr#1493, to-chk=2495/5944)
labels/train/16fa10c7-Ch120241207124102.txt
          1.72K 100%    1.30kB/s    0:00:01 (xfr#1494, to-chk=2494/5944)
labels/train/16fe91fc-Ch120241207124118.txt
          1.71K 100%    0.00kB/s    0:00:00 (xfr#1495, to-chk=2493/5944)
labels/train/172768a1-Ch120250107014033.txt
            986 100%    1.58kB/s    0:00:00 (xfr#1496, to-chk=2492/5944)
labels/train/1741b919-Ch120241207221956.txt
            899 100%    0.67kB/s

In [10]:
model_path = "/content/drive/MyDrive/yolov8s.pt"
# YAMLファイルのパス
data_yaml = "/content/dataset/data.yaml"

In [11]:
import yaml


# YAMLファイルを読み込む
with open(data_yaml, "r") as file:
    data = yaml.safe_load(file)

# 変更したいパス
data["test"] = "./images/test"
data["train"] = "./images/train"
data["val"] = "./images/val"

# YAMLファイルを書き戻す
with open(data_yaml, "w") as file:
    yaml.safe_dump(data, file, default_flow_style=False)

print("data.yaml のパスを更新しました。")

data.yaml のパスを更新しました。


In [ ]:
import os
from ultralytics import YOLO
import numpy as np


def train_yolov8(model_path, data_yaml, epochs, batch_size, img_size, output_dir, iou_threshold, mosaic, mixup):
    """
    Train YOLOv8 model with enhanced settings for small overlapping object detection.
    """
    print(f"Loading model from {model_path}")
    model = YOLO(model_path)

    # Train the model with optimized settings
    print(f"Training YOLOv8 with {epochs} epochs...")
    model.train(
        data=data_yaml,
        epochs=epochs,
        batch=batch_size,
        imgsz=img_size,
        project=output_dir,
        name="yolov8_training",
        iou=iou_threshold,  # Adjust IoU threshold for better object separation
        mosaic=mosaic,  # Enhance data augmentation with mosaic images
        mixup=mixup,  # Mixup augmentation for diverse training
        degrees=180,  # Rotation augmentation
    )

    print(f"Training complete. Results saved to {output_dir}")

# Colab 用にパラメータを手動で指定
model_path = "/content/drive/MyDrive/yolov8s.pt"  # モデルのパス (Google Drive に保存している場合は適宜変更)
#data_yaml = "/content/drive/MyDrive/MaiNDLab/dataset/data.yaml"  # データセットの YAML ファイル
epochs = 200  # エポック数
batch_size = 32  # バッチサイズ
img_size = 960  # 画像サイズ
iou_threshold = 0.3  # IoU 閾値
mosaic = 1.0  # モザイクの確率
mixup = 0.2  # Mixup の確率
output_dir = "runs_experiment"  # 出力ディレクトリ

# 出力ディレクトリを作成
os.makedirs(output_dir, exist_ok=True)

# YOLOv8 の学習実行
train_yolov8(
    model_path=model_path,
    data_yaml=data_yaml,
    epochs=epochs,
    batch_size=batch_size,
    img_size=img_size,
    output_dir=output_dir,
    iou_threshold=iou_threshold,
    mosaic=mosaic,
    mixup=mixup
)


Loading model from /content/drive/MyDrive/yolov8s.pt
Training YOLOv8 with 200 epochs...
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/yolov8s.pt, data=/content/dataset/data.yaml, epochs=200, time=None, patience=100, batch=32, imgsz=960, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs_experiment, name=yolov8_training4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.3, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mas

train: Scanning /content/dataset/labels/train.cache... 2076 images, 0 backgrounds, 1 corrupt: 100%|██████████| 2076/2076 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/dataset/images/train/4e6a8c0e-Ch120241207152724.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0725]
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/dataset/labels/val.cache... 593 images, 0 backgrounds, 0 corrupt: 100%|██████████| 593/593 [00:00<?, ?it/s]


Plotting labels to runs_experiment/yolov8_training4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000149, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 960 train, 960 val
Using 8 dataloader workers
Logging results to runs_experiment/yolov8_training4
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

     13/200      23.5G      1.172     0.7886      1.269        943        960: 100%|██████████| 65/65 [00:24<00:00,  2.68it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:05<00:00,  1.87it/s]


                   all        593      10600      0.911      0.906      0.921      0.608

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     14/200      23.5G      1.159     0.7574       1.26       1337        960: 100%|██████████| 65/65 [00:25<00:00,  2.53it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:04<00:00,  2.28it/s]


                   all        593      10600      0.907      0.898      0.929      0.618

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     15/200      19.5G      1.143     0.7398      1.245       1405        960:  55%|█████▌    | 36/65 [00:12<00:16,  1.75it/s]